### Install OpenAI Library

if this is your first time using OpenAI library, you must first install OpenAI

In [ ]:
!pip install --upgrade pip

: 

In [ ]:
!pip install -U openai

### Import Libraries

Import openai and json libraries as we will use them in our code

In [1]:
import openai
import json
import re
import pandas as pd

### Movie Database

This file contains a list of movies collected from Movie Lens database.  
The data includes movie lens, TMDb, IMDb information as well as average user ratings by Movie Lens users

In [2]:
df_movie_abt = pd.read_csv("https://raw.githubusercontent.com/drarvindsathi/movie_recommender_data/main/data/movie_abt_with_rating.csv", dtype='unicode')
movie_json_list = json.loads(df_movie_abt.to_json(orient="records"))
df_movie_abt


,Unnamed: 0,Unnamed: 0.1,movieId,title,genres,movie_title,movie_year,movie_genre,imdbId,tmdbId,...,vote_count,tid,title_IMDB,url,imdbRating,duration,year,nrOfWins,nrOfNominations,rating
0,0,0,1.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,"['Adventure', 'Animation', 'Children', 'Comedy...",tt0114709,862.0,...,5269.0,tt0114709,Toy Story (1995),http://www.imdb.com/title/tt0114709/,8.3,4860.0,1995.0,19,13.0,3.921239561324077
1,1,1,2.0,Jumanji (1995),Adventure|Children|Fantasy,Jumanji,1995,"['Adventure', 'Children', 'Fantasy']",tt0113497,8844.0,...,NaN,tt0113497,Jumanji (1995),http://www.imdb.com/title/tt0113497/,6.8,6240.0,1995.0,4,8.0,3.2119768016904193
2,2,2,3.0,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men,1995,"['Comedy', 'Romance']",tt0113228,15602.0,...,NaN,tt0113228,Der dritte Fr√ºhling (1995),http://www.imdb.com/title/tt0113228/,6.5,6060.0,1995.0,2,1.0,3.1510404397330194
3,3,3,4.0,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale,1995,"['Comedy', 'Drama', 'Romance']",tt0114885,31357.0,...,NaN,tt0114885,Waiting to Exhale - Warten auf Mr. Right (1995),http://www.imdb.com/title/tt0114885/,5.5,7440.0,1995.0,8,8.0,2.8613933236574747
4,4,4,5.0,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II,1995,['Comedy'],tt0113041,11862.0,...,NaN,tt0113041,Ein Geschenk des Himmels - Vater der Braut 2 (...,http://www.imdb.com/title/tt0113041/,5.8,6360.0,1995.0,1,1.0,3.064591727653976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38783,38783,38783,NaN,NaN,NaN,Peep Show Sectioning,2005,NaN,NaN,NaN,...,NaN,tt0672488,Peep Show Sectioning (TV Episode 2005),http://www.imdb.com/title/tt0672488/,7.7,1440,2005,0,0,0.0
38784,38784,38784,NaN,NaN,NaN,Playhouse 90 The Miracle Worker,1957,NaN,NaN,NaN,...,NaN,tt0675644,Playhouse 90 The Miracle Worker (TV Episode 1957),http://www.imdb.com/title/tt0675644/,7.3,5400,1957,0,0,0.0
38785,38785,38785,NaN,NaN,NaN,Private Screenings Robert Mitchum and Jane Rus...,1996,NaN,NaN,NaN,...,NaN,tt0679222,Private Screenings Robert Mitchum and Jane Rus...,http://www.imdb.com/title/tt0679222/,7,3600,1996,0,0,0.0
38786,38786,38786,NaN,NaN,NaN,Providence All the King's Men,2002,NaN,NaN,NaN,...,NaN,tt0680064,Providence All the King's Men (TV Episode 2002),http://www.imdb.com/title/tt0680064/,NaN,3600,2002,0,0,0.0


In [3]:
df_movie_abt.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'movieId', 'title', 'genres',
       'movie_title', 'movie_year', 'movie_genre', 'imdbId', 'tmdbId',
       'budget', 'genres_TMDB', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline',
       'title_TMDB', 'vote_average', 'vote_count', 'tid', 'title_IMDB', 'url',
       'imdbRating', 'duration', 'year', 'nrOfWins', 'nrOfNominations',
       'rating'],
      dtype='object')

### Function to get movie information

The following function will find movie information in response to a specific query.  The function uses a movie repository using TMDb, IMDb and Movie Lens information and has included in it user rating from Movie Lens viewers

In [4]:
def year_in_decade(year, decade):
    decade_chars = len(decade)
    start_of_decade = int(decade[0:decade_chars-1])
    end_of_decade = start_of_decade +10
    if year:
        if len(year) == 4:
            if int(year) >= start_of_decade:
                if int(year) < end_of_decade:
                    return True
    return False

def list_of_genres(movie_json_list):
    genre_list = []
    for movie_json in movie_json_list:
        if movie_json['genres']:
            movie_genre_list = movie_json['genres'].split('|')
            for genre in movie_genre_list:
                if genre not in genre_list:
                    genre_list.append(genre)
    return genre_list

def provide_movie_details(dict, movie_json_list):
    intent = dict["intent"]
    entity_type = dict["entity_type_3"]
    entity_value = dict["entity_value_3"]
    if intent == 'provide-movie-details':
        if entity_type == 'movie':
            for movie_json in movie_json_list:
                if movie_json['movie_title']:
                    movie_title = movie_json['movie_title']
                    if entity_value in movie_title:
                        length = movie_json['runtime']
                        year = movie_json['movie_year']
                        budget = movie_json['budget']
                        genre = movie_json['genres']
                        rating = movie_json['rating']
                        sent_1 = "%s is %s min long, from year %s, made with a budget of %s." % (entity_value, length, year, budget)
                        sent_2 = " Its genres include %s. It received a rating of %s from our viewers." % (genre, rating)
                        return sent_1 + sent_2



def recommend(dict):
    intent = dict["intent"]

    if intent == 'list-of-genres':
        genre_list = list_of_genres(movie_json_list)
        number_of_genres = len(genre_list)
        dict['message'] = [number_of_genres, genre_list]
        return dict


    if intent == 'count-movies-by-genre':
        entity_type = dict["entity_type_1"]
        entity_value = dict["entity_value_1"]
        if entity_type == "genre":
            genre_to_count = entity_value
            movie_in_genre = []
            for movie_json in movie_json_list:
                if movie_json['movie_genre']:
                    if genre_to_count in movie_json["movie_genre"]:
                        movie_in_genre.append(movie_json)
            dict['message'] = len(movie_in_genre)
            return dict

    if intent == 'provide-movie-details':
        dict['message'] = provide_movie_details(dict, movie_json_list)
        return dict

    if intent == "recommend-movie-by-decade":
        entity_type_1 = dict["entity_type_1"]
        entity_value_1 = dict["entity_value_1"]
        entity_type_2 = dict["entity_type_2"]
        entity_value_2 = dict["entity_value_2"]
        if entity_type_1 == "genre":
            genre_to_search = entity_value_1
            if entity_type_2 == "decade":
                decade_to_search = entity_value_2
                movie_in_genre_decade = []
                for movie_json in movie_json_list:
                    if movie_json['movie_genre']:
                        if genre_to_search in movie_json["movie_genre"]:
                            if year_in_decade(movie_json["movie_year"], decade_to_search):
                                movie_in_genre_decade.append(movie_json)
                if movie_in_genre_decade:
                    movie_in_genre_decade.sort(key=lambda x: x["rating"], reverse=True)
                    dict['message'] = movie_in_genre_decade[0]["title"]
                    return dict
                else:
                    dict['message'] = "I could not find a match"
                    return dict
            else:
                dict['message'] = "you did not specify decade"
                return dict
        else:
            dict['message']: "you did not specify genre"
            return dict


In [5]:
example_1 = {"intent": "count-movies-by-genre",
             "entity_type_1": "genre",
             "entity_value_1": "Action"}

recommend(example_1)

{'intent': 'count-movies-by-genre',
 'entity_type_1': 'genre',
 'entity_value_1': 'Action',
 'message': 3520}

In [6]:
example_2 = {"intent": "recommend-movie-by-decade",
             "entity_type_1": "genre",
             "entity_value_1": "Action",
             "entity_type_2": "decade",
             "entity_value_2": "1970s"}
recommend(example_2)

{'intent': 'recommend-movie-by-decade',
 'entity_type_1': 'genre',
 'entity_value_1': 'Action',
 'entity_type_2': 'decade',
 'entity_value_2': '1970s',
 'message': 'Deewaar (1975)'}

In [7]:
example_3 = {"intent": "provide-movie-details",
             "entity_type_3": "movie",
             "entity_value_3": "Toy Story"}
recommend(example_3)

{'intent': 'provide-movie-details',
 'entity_type_3': 'movie',
 'entity_value_3': 'Toy Story',
 'message': 'Toy Story is 81.0 min long, from year 1995, made with a budget of 30000000.0. Its genres include Adventure|Animation|Children|Comedy|Fantasy. It received a rating of 3.921239561324077 from our viewers.'}

### Get an API Key

To subscribe to OpenAI and obtain an API key (also known as a secret key), you can follow these steps:

1. Go to the OpenAI website: https://chat.openai.com/auth/login
2. Click on the "Sign up" button at the top-right corner of the page.
3. Follow the instructions to create an account, which will involve providing your name, email address, and creating a password.
4. Once you have created an account, you will be taken to your dashboard. Click on the "API Keys" link in the left sidebar.
5. On the API Keys page, click the "Generate API Key" button.
6. You will be prompted to enter a name for your API key (e.g. "My Test Key"). Enter a descriptive name and click the "Generate" button.
7. Your API key will be generated and displayed on the page. Make sure to copy the key and store it in a safe place.

Note that OpenAI currently charges for API usage beyond a certain amount of free usage per month, so make sure to review the pricing and billing information on their website if you plan to use the API for commercial purposes.  If you use the key for exercise in our course, it will be within their free use.


### Use api key for your model

Create JSON file config.json using the format:
{"secret_key": "<your secret key>"}

where you will replace <your secret key> with the API key you obtained from OpenAI.   

In [8]:
with open("config.json") as json_file:
    config = json.load(json_file)
openai.api_key = config['secret_key']

### Create Your GPT-4 Conversation for Intent Detection

We will initiate a conversation log to define the role as the front end that detects intents, extract entities and use prompts to accomplish that task.

### Log your conversations

GPT-4 can deal with a conversation dialog with multiple interactions.  
To facilitate, we will create a JSON that will store your conversations and seed with an open statement.
We will use this JSON to store conversations initiated by the system and user
using 'role' with values 'system' and 'user'

In [9]:
conversations = []

In [10]:
conversations.append({'role': 'system', 'content': """Your role is to detect intents and extract entities in 
user queries.  The output is a JSON which will be used to query a backend system"""})

In [11]:
print('{0}: {1}\n'.format(conversations[-1]['role'].strip(), conversations[-1]['content'].strip()))

system: Your role is to detect intents and extract entities in 
user queries.  The output is a JSON which will be used to query a backend system



### Add a user querry

Now that we have created an opening statement, let's add a user query by appending to the conversations JSON.

In [12]:
user_query = "Do you have a 1970s action movie?"

In [13]:
prompt = """Use the following examples for intent definition 
Example: How many Genres do you have -> Intent: list-of-genres 
Example: How many action movies do you have -> Intent: count-movies-by-genre 
Example: Can you recommend a 1990s action movie -> Intent: recommend-movie-by-decade 
Example: Can you provide Toy Story movie details -> Intent: provide-movie-details 
Query: %s Please provide the intents in JSON format {'intent': '<detected intent>'} which can be directly 
used in a backend query""" % (user_query)


In [14]:
conversations.append({"role": "user", "content": prompt})
conversations

[{'role': 'system',
  'content': 'Your role is to detect intents and extract entities in \nuser queries.  The output is a JSON which will be used to query a backend system'},
 {'role': 'user',
  'content': "Use the following examples for intent definition \nExample: How many Genres do you have -> Intent: list-of-genres \nExample: How many action movies do you have -> Intent: count-movies-by-genre \nExample: Can you recommend a 1990s action movie -> Intent: recommend-movie-by-decade \nExample: Can you provide Toy Story movie details -> Intent: provide-movie-details \nQuery: Do you have a 1970s action movie? Please provide the intents in JSON format {'intent': '<detected intent>'} which can be directly \nused in a backend query"}]

### Use GPT-4 to detect Intents and extract entities

Here we will use gpt-4 to detect intent and entities from user queries


In [15]:
response = openai.ChatCompletion.create(
  model="gpt-4",
  temperature=0,
  messages=conversations
)


In [16]:
conversations.append({'role': response.choices[0].message.role, 
                      'content': response.choices[0].message.content.strip()})

### Print conversations

Let's print all conversations so far...

In [17]:
for conversation in conversations:
    print('{0}: {1}\n'.format(conversation['role'].strip(), conversation['content'].strip()))

system: Your role is to detect intents and extract entities in 
user queries.  The output is a JSON which will be used to query a backend system

user: Use the following examples for intent definition 
Example: How many Genres do you have -> Intent: list-of-genres 
Example: How many action movies do you have -> Intent: count-movies-by-genre 
Example: Can you recommend a 1990s action movie -> Intent: recommend-movie-by-decade 
Example: Can you provide Toy Story movie details -> Intent: provide-movie-details 
Query: Do you have a 1970s action movie? Please provide the intents in JSON format {'intent': '<detected intent>'} which can be directly 
used in a backend query

assistant: {
  "intent": "recommend-movie-by-decade",
  "entities": {
    "decade": "1970s",
    "genre": "action"
  }
}



### User query to extract entities

Here we will use gpt-4 to extract entities from user queries.  The response is appened to the conversations

In [18]:
prompt = """Use the following examples for entity definition
Example: How many action movies do you have  -> entity_type_1: genre, entity_value_1: Action
Example: Can you recommend a 1990s drama movie -> entity_type_1: genre, entity_value_1: Drama, 
             entity_type_2: decade, entity_value_2: 1990s
Example: Can you provide Toy Story movie details -> entity_type_3: movie, entity_value_3: Toy Story 2
Query: %s revise JSON with intent to include 
                            {"intent": <intent in the JSON>,
                             "entity_type_1": "genre",
                             "entity_value_1": "<genre found in query with first letter capitalized">, 
                             "entity_type_2": "decade", 
                             "entity_value_2": <decade information>,
                             "entity_type_3": "movie", "entity_value_3": <movie information found in the query>}. 
Your role is to provide JSON which will be used in backend query
""" % (user_query)

In [19]:
prompt

'Use the following examples for entity definition\nExample: How many action movies do you have  -> entity_type_1: genre, entity_value_1: Action\nExample: Can you recommend a 1990s drama movie -> entity_type_1: genre, entity_value_1: Drama, \n             entity_type_2: decade, entity_value_2: 1990s\nExample: Can you provide Toy Story movie details -> entity_type_3: movie, entity_value_3: Toy Story 2\nQuery: Do you have a 1970s action movie? revise JSON with intent to include \n                            {"intent": <intent in the JSON>,\n                             "entity_type_1": "genre",\n                             "entity_value_1": "<genre found in query with first letter capitalized">, \n                             "entity_type_2": "decade", \n                             "entity_value_2": <decade information>,\n                             "entity_type_3": "movie", "entity_value_3": <movie information found in the query>}. \nYour role is to provide JSON which will be used in 

In [20]:
conversations.append({"role": "user", "content": prompt})

### Use GPT-4 to extract entities

Here we will use gpt-4 to extract entities from user queries

In [21]:
response = openai.ChatCompletion.create(
  model="gpt-4",
  temperature=0,
  messages=conversations
)


In [22]:
conversations.append({'role': response.choices[0].message.role, 
                      'content': response.choices[0].message.content.strip()})

### Print conversations

Let's print all conversations so far...

In [23]:
for conversation in conversations:
    print('{0}: {1}\n'.format(conversation['role'].strip(), conversation['content'].strip()))

system: Your role is to detect intents and extract entities in 
user queries.  The output is a JSON which will be used to query a backend system

user: Use the following examples for intent definition 
Example: How many Genres do you have -> Intent: list-of-genres 
Example: How many action movies do you have -> Intent: count-movies-by-genre 
Example: Can you recommend a 1990s action movie -> Intent: recommend-movie-by-decade 
Example: Can you provide Toy Story movie details -> Intent: provide-movie-details 
Query: Do you have a 1970s action movie? Please provide the intents in JSON format {'intent': '<detected intent>'} which can be directly 
used in a backend query

assistant: {
  "intent": "recommend-movie-by-decade",
  "entities": {
    "decade": "1970s",
    "genre": "action"
  }
}

user: Use the following examples for entity definition
Example: How many action movies do you have  -> entity_type_1: genre, entity_value_1: Action
Example: Can you recommend a 1990s drama movie -> enti

### Invoke back end function 

Using the JSON created by GPT-4, now you can invoke the backend function

In [24]:
response.choices[0].message.content.strip()

'{\n  "intent": "recommend-movie-by-decade",\n  "entity_type_1": "genre",\n  "entity_value_1": "Action",\n  "entity_type_2": "decade",\n  "entity_value_2": "1970s",\n  "entity_type_3": "movie",\n  "entity_value_3": ""\n}'

In [25]:
backend_query_str = response.choices[0].message.content.strip()
backend_query = json.loads(backend_query_str)
backend_query

{'intent': 'recommend-movie-by-decade',
 'entity_type_1': 'genre',
 'entity_value_1': 'Action',
 'entity_type_2': 'decade',
 'entity_value_2': '1970s',
 'entity_type_3': 'movie',
 'entity_value_3': ''}

In [26]:
backend_query_result = recommend(backend_query)

In [27]:
json.dumps(backend_query_result)

'{"intent": "recommend-movie-by-decade", "entity_type_1": "genre", "entity_value_1": "Action", "entity_type_2": "decade", "entity_value_2": "1970s", "entity_type_3": "movie", "entity_value_3": "", "message": "Deewaar (1975)"}'

### Response to User

Once the backend query provides the response, we can ask GPT-4 to construct a sentence.
GPT-4 will convert the backend response in JSON format to a sentence that can be displayed to user

In [28]:
conversations = []

In [29]:
conversations.append({"role": "user", 
                      "content": "Response to query in JSON form " + 
                                 json.dumps(backend_query_result) + 
        " can you refer to the original user query and convert this JSON into English response to user"})

In [30]:
response = openai.ChatCompletion.create(
  model="gpt-4",
  temperature=1,
  messages=conversations
)

In [31]:
conversations.append({'role': response.choices[0].message.role, 
                      'content': response.choices[0].message.content.strip()})

In [32]:
for conversation in conversations:
    print('{0}: {1}\n'.format(conversation['role'].strip(), conversation['content'].strip()))

user: Response to query in JSON form {"intent": "recommend-movie-by-decade", "entity_type_1": "genre", "entity_value_1": "Action", "entity_type_2": "decade", "entity_value_2": "1970s", "entity_type_3": "movie", "entity_value_3": "", "message": "Deewaar (1975)"} can you refer to the original user query and convert this JSON into English response to user

assistant: Based on your preferences for action movies from the 1970s, I recommend watching "Deewaar (1975)".



### Orchestrator

Let's use the code created in this Notebook to define an orchestrator
It will perform the following functions:
- It will use a welcoming statement to get the user started
- It will interpret the query from user to find user intent and entities
- If the user has made an incomplete query, it will use disambiguation to complete the intent and entities
- It will engage the recommendation engine to get a response
- It will use chatGPT to get response to a question in case intent is not covered by the recommendation engine
- It will inform back the user and ask if there are additional questions

In [33]:
global CONVERSATIONS
global BACK_END_CONVERSATIONS

def find_intents_entities(user_query):
    
    global BACK_END_CONVERSATIONS
    
    BACK_END_CONVERSATIONS.append({'role': 'system', 'content': """Your role is to detect intents and extract entities in 
user queries.  The output is a JSON which will be used to query a backend system"""})
    
    prompt_intent = """Use the following examples for intent definition 
                    Example: How many Genres do you have -> Intent: list-of-genres 
                    Example: How many action movies do you have -> Intent: count-movies-by-genre 
                    Example: Can you recommend a 1990s action movie -> Intent: recommend-movie-by-decade 
                    Example: Can you provide Toy Story movie details -> Intent: provide-movie-details 
                    Query: %s Please provide the intents in JSON format 
                    {'intent': '<detected intent>'} 
                    which can be directly used in a backend query""" % (user_query)
    
    BACK_END_CONVERSATIONS.append({"role": "user", "content": prompt_intent})
    
    response = openai.ChatCompletion.create(model="gpt-4",
                                            temperature=0,
                                            messages=BACK_END_CONVERSATIONS)
    
    BACK_END_CONVERSATIONS.append({'role': response.choices[0].message.role, 
                                   'content': response.choices[0].message.content.strip()})
    
    backend_query_str = response.choices[0].message.content.strip()
#    print(backend_query_str)
    query_json = json.loads(backend_query_str)
    if query_json['intent'] in ['count-movies-by-genre', 'recommend-movie-by-decade']:
    
        prompt_entities = """Use the following examples for entity definition
                          Example: How many action movies do you have  -> entity_type_1: genre, entity_value_1: Action
                          Example: Can you recommend a 1990s drama movie -> entity_type_1: genre, entity_value_1: Drama, 
                           entity_type_2: decade, entity_value_2: 1990s
                          Example: Can you provide Toy Story movie details -> entity_type_3: movie, entity_value_3: Toy Story 2
                          Query: %s revise JSON with intent to include 
                            {"intent": <intent in the JSON>,
                             "entity_type_1": "genre",
                             "entity_value_1": "<genre found in query with first letter capitalized">, 
                             "entity_type_2": "decade", 
                             "entity_value_2": <decade information>,
                             "entity_type_3": "movie", "entity_value_3": <movie information found in the query>}.  
                          Your role is to provide JSON which will be used in backend query""" % (user_query)
#        print(prompt_entities)
    
        BACK_END_CONVERSATIONS.append({"role": "user", "content": prompt_entities})
    
        response = openai.ChatCompletion.create(model="gpt-4",
                                                temperature=0,
                                            messages=BACK_END_CONVERSATIONS)
#        print(response)

        BACK_END_CONVERSATIONS.append({'role': response.choices[0].message.role, 
                          'content': response.choices[0].message.content.strip()})
    
        backend_query_str = response.choices[0].message.content.strip()
        query_json = json.loads(backend_query_str)
    
    return query_json


def orchestrator(dialog):
    
    global CONVERSATIONS
    global BACK_END_CONVERSATIONS
    
    CONVERSATIONS.append({'role': 'system', 
                          'content': """Welcome to the movie recommendation system.  
    We have 40,000 movies and 500K ratings in our recommendation system.
    Which movie would you like to watch.  Please tell us genre, and decade?"""})
    for conversation in CONVERSATIONS:
        print(conversation['role'], " : ", conversation['content'])


    while dialog == True:
        
        user_query = input("Please enter your query: ")
        
        CONVERSATIONS.append({'role': 'user', 
                              'content': user_query})
    
     #   for conversation in CONVERSATIONS:
     #       print(conversation['role'], " : ", conversation['content'])
    
        
        BACK_END_CONVERSATIONS = []
        
  #      print(user_query)
    
        query_json = find_intents_entities(user_query)
        
 #       print(query_json)
    
        backend_query_result = recommend(query_json)
        
        
        CONVERSATIONS.append({"role": "system", 
                      "content": "Response to query in JSON form " + 
                                 json.dumps(backend_query_result) + 
        " refer to original user query and convert JSON into English response with movie overview or genre definitions using a chatty, cheerful persona"})
        
        response = openai.ChatCompletion.create(model="gpt-4",
                                                temperature=1,
                                                messages=CONVERSATIONS)
        
        CONVERSATIONS.append({'role': response.choices[0].message.role, 
                      'content': response.choices[0].message.content.strip()})
        
        print(response.choices[0].message.role, " : ", response.choices[0].message.content.strip())
        
        #for conversation in CONVERSATIONS:
        #    print(conversation['role'], " : ", conversation['content'])
        
    
        user_response = input("\nDo you have additional questions? (Yes/No): ")
        
        if user_response.lower() == 'no':
            print("Thank you for using the Orchestrator. Have a great day!")
            dialog = False
        else:
            dialog = True


### Example using Orchestrator

Here we will run the orchestrator

In [34]:
CONVERSATIONS = []
dialog = True
orchestrator(dialog)

system  :  Welcome to the movie recommendation system.  
    We have 40,000 movies and 500K ratings in our recommendation system.
    Which movie would you like to watch.  Please tell us genre, and decade?
Please enter your query: How many genres do you have in your database?
assistant  :  We have 20 exciting genres in our database for you to explore! Here's a list of them:

1. Adventure: Movies filled with exciting journeys and daring exploits.
2. Animation: Films brought to life using various animation techniques.
3. Children: Wholesome entertainment suitable for kids.
4. Comedy: Movies that'll make you laugh and lift your spirits.
5. Fantasy: Films set in magical realms or featuring mythical creatures.
6. Romance: Love stories that'll tug at your heartstrings.
7. Drama: Engaging narratives that capture the essence of human emotions.
8. Action: Fast-paced flicks packed with adrenaline-pumping sequences.
9. Crime: Thrilling plots involving lawbreakers and their pursuers.
10. Thriller: